In [1]:
import pandas as pd
import numpy as np
labels = pd.read_csv("/Users/mingliangang/Desktop/cifar10/trainLabels.csv")
labels.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [2]:
label2id = {x:i for i,x in enumerate(list(set(labels["label"])))}
id2label = {i:x for i,x in enumerate(list(set(labels["label"])))}
labelcon = lambda x: label2id[x]
labels['label'] = labels['label'].map(labelcon)
labels_ = labels['label'].as_matrix()

In [3]:
labels['label'].head()

0    9
1    5
2    5
3    6
4    8
Name: label, dtype: int64

In [4]:
labels_

array([9, 5, 5, ..., 5, 8, 8])

# Sanity check

In [5]:
labels['label'].map(lambda x: id2label[x]).head()

0          frog
1         truck
2         truck
3          deer
4    automobile
Name: label, dtype: object

# Tensorflow input pipeline

In [6]:
import os
from sklearn.model_selection import train_test_split
import cv2
train = np.array([cv2.imread("/Users/mingliangang/Desktop/cifar10/train/"+i) for i in os.listdir("/Users/mingliangang/Desktop/cifar10/train")]).astype(np.float32)
one_hot = np.zeros((labels_.shape[0], 10))
one_hot[np.arange(labels_.shape[0]), labels_] = 1
X_train, X_val, y_train, y_val = train_test_split(train, one_hot, test_size=0.33, random_state=42)

# Resnet Model

In [7]:
from models import resnet
import tensorflow as tf

batch_size = 128
GPU = True
summaries_dir = "/Users/mingliangang/Desktop/resnet2/"

if GPU:
    with tf.device('/device:GPU:0'):
        X = tf.placeholder("float", [batch_size, 32, 32, 3])
        Y = tf.placeholder("float", [batch_size, 10])
        learning_rate = tf.placeholder("float", [])
        net = resnet(X,20)
        cross_entropy = -tf.reduce_sum(Y*tf.log(net))
        tf.summary.scalar('cross_entropy', cross_entropy)
        opt = tf.train.MomentumOptimizer(learning_rate, 0.9,use_nesterov=True)
        train_op = opt.minimize(cross_entropy)
        
    merged = tf.summary.merge_all()
    config = tf.ConfigProto(allow_soft_placement = True)
    sess = tf.Session(config = config)
    writer = tf.summary.FileWriter(summaries_dir + '/gpu', sess.graph)
    sess.run(tf.global_variables_initializer())
    
else:
    with tf.device('/device:CPU:0'):
        X = tf.placeholder("float", [batch_size, 32, 32, 3])
        Y = tf.placeholder("float", [batch_size, 10])
        learning_rate = tf.placeholder("float", [])
        net = resnet(X,20)
        cross_entropy = -tf.reduce_sum(Y*tf.log(net))
        tf.summary.scalar('cross_entropy', cross_entropy)
        opt = tf.train.MomentumOptimizer(learning_rate, 0.9,use_nesterov=True)
        train_op = opt.minimize(cross_entropy)
        
    merged = tf.summary.merge_all()
    sess = tf.Session()
    writer = tf.summary.FileWriter(summaries_dir + '/cpu', sess.graph)
    sess.run(tf.global_variables_initializer())

Tensor("Shape:0", shape=(4,), dtype=int32, device=/device:GPU:0)


In [8]:
#Multiple epochs
from tqdm import tqdm
n_epochs = 1
for _ in tqdm(range(n_epochs)):
  i = 0
  while True:
    try:
      x_batch = X_train[i*batch_size:(i+1)*batch_size]
      y_batch = y_train[i*batch_size:(i+1)*batch_size]
      summary,loss,_ = sess.run([merged,cross_entropy,train_op], feed_dict={X: x_batch, Y: y_batch,learning_rate: 0.001})
      writer.add_summary(summary, i)
      i+=1
    except ValueError:
      print(loss)
      break

100%|██████████| 1/1 [00:09<00:00,  9.15s/it]

295.189
